#Обучение модели на русских текстах, содержащих эмодзи

In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 10.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=afcf880bc7a6d9d6d5a812236504533fe07e30193bc413512db49c562a895b2d
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [2]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 4.3 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=1a70f52293708bda25f867be7a99c0884d3aaff1aee5aa0ec7aea4f05c1162da
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##**Подготовка датасета со смайликами**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/maxim/marking.csv')

In [ ]:
data = data.drop(columns=['image_id', 'name', 'group', 'sub_group', 'train'])

In [ ]:
data

,text,emoji
0,Ухмыляющееся лицо,😀
1,Ухмыляющееся лицо с большими глазами,😃
2,Улыбающееся лицо с улыбающимися глазами,😄
3,Сияющее лицо с улыбающимися глазами,😁
4,Ухмыляющееся щурящееся лицо,😆
...,...,...
2744,Флаг: Замбия,🇲🇿
2745,Флаг: Зимбабве,🇿🇼
2746,Флаг: Англия,🏴󠁧󠁢󠁥󠁮󠁧󠁿
2747,Флаг: Шотландия,🏴󠁧󠁢󠁳󠁣󠁴󠁿


In [ ]:
regular = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
full_tokens = []
for text in data['text']:
  text = re.sub(regular, '', text)
  tokens = word_tokenize(text, language='russian')
  full_tokens.append(tokens)
data['tokens'] = full_tokens

In [ ]:
data = data.drop('text', axis = 1)

In [ ]:
data

,emoji,tokens
0,😀,"[Ухмыляющееся, лицо]"
1,😃,"[Ухмыляющееся, лицо, с, большими, глазами]"
2,😄,"[Улыбающееся, лицо, с, улыбающимися, глазами]"
3,😁,"[Сияющее, лицо, с, улыбающимися, глазами]"
4,😆,"[Ухмыляющееся, щурящееся, лицо]"
...,...,...
2744,🇲🇿,"[Флаг, Замбия]"
2745,🇿🇼,"[Флаг, Зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[Флаг, Англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[Флаг, Шотландия]"


In [ ]:
filtered = []
morph = pymorphy2.MorphAnalyzer()
for tokens in data['tokens']:
  filtered_tokens = []
  for token in tokens:
    token = morph.normal_forms(token)[0]
    if token not in stopwords.words('russian'):
      filtered_tokens.append(token)
  filtered.append(filtered_tokens)

In [ ]:
data['filtered'] = filtered

In [ ]:
data = data.drop('tokens', axis = 1)

In [ ]:
data

,emoji,filtered
0,😀,"[ухмыляться, лицо]"
1,😃,"[ухмыляться, лицо, больший, глаз]"
2,😄,"[улыбаться, лицо, улыбаться, глаз]"
3,😁,"[сиять, лицо, улыбаться, глаз]"
4,😆,"[ухмыляться, щуриться, лицо]"
...,...,...
2744,🇲🇿,"[флаг, замбия]"
2745,🇿🇼,"[флаг, зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[флаг, англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[флаг, шотландия]"


In [ ]:
for emoji in data['emoji']:
  ind = data[data['emoji'] == emoji].index
  ind.to_list()
  data.at[ind[0], 'filtered'].append(emoji)

In [ ]:
data = data.drop('emoji', axis = 1)

In [ ]:
data

,filtered
0,"[ухмыляться, лицо, 😀]"
1,"[ухмыляться, лицо, больший, глаз, 😃]"
2,"[улыбаться, лицо, улыбаться, глаз, 😄]"
3,"[сиять, лицо, улыбаться, глаз, 😁]"
4,"[ухмыляться, щуриться, лицо, 😆]"
...,...
2744,"[флаг, замбия]"
2745,"[флаг, зимбабве, 🇿🇼]"
2746,"[флаг, англия, 🏴󠁧󠁢󠁥󠁮󠁧󠁿]"
2747,"[флаг, шотландия, 🏴󠁧󠁢󠁳󠁣󠁴󠁿]"


##**Подготовка датасета с комментариями ВК**

In [6]:
colnames = ['index', 'comment']

In [7]:
comments = pd.read_csv('/content/drive/MyDrive/maxim/comments.csv', names=colnames, header=None)

In [8]:
comments = comments.drop(labels = [0,1],axis = 0)

In [9]:
comments = comments.drop(columns = ['index'],axis = 1)

In [10]:
comments

,comment
2,а в чём прикол дипломатов туда-сюда гонять ?
3,"Вообще, это все логично. Абсолютно ожидаемое п..."
4,NaN
5,"[id717273538|Matilda], детишками ты никого не ..."
6,"Складывается ощущение, что за всё отвечает зах..."
...,...
10100,"[id11875095|Сергей], даа, особенно после истор..."
10101,"[id155404424|Ruslan], про пытки в тюрьмах напо..."
10102,"[id11875095|Sergey], кто эти вы, которые туда ..."
10103,Это импорто замещения


In [11]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc))
    for c in str(doc):
      if emoji.is_emoji(c):
          i = doc.index(c)
          doc = doc[:i] + doc[i:]
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru and emoji.is_emoji(token) == False:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
        elif emoji.is_emoji(token):
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [12]:
comments = comments['comment'].apply(lemmatize)

In [13]:
comments = comments.dropna()

In [14]:
comments

2              [что, прикол, дипломат, туда, сюда, гонять]
3        [вообще, это, логично, абсолютно, ожидать, пов...
5        [детишки, никто, изжалобить, весь, мир, против...
6         [складываться, ощущение, всё, отвечать, захаров]
8        [«никакий, правительство, мир, смочь, проводит...
                               ...                        
10100    [сергей, даа, особенно, история, флойдом, прав...
10101    [пытка, тюрьма, напомнить, или, неудобно, гово...
10102    [который, туда, стремиться, любой, страна, ник...
10103                            [это, импорто, замещение]
10104          [для, чумазый, это, норма, лоток, приучить]
Name: comment, Length: 8718, dtype: object

Проверка на наличие эмодзи

In [17]:
def check(comment):
  flag=0
  for item in comment:
    if emoji.is_emoji(item):
      flag+=1
  if flag > 0:
    return(comment)
  return None
  

In [18]:
emojis = comments.apply(check)

In [19]:
emojis = emojis.dropna()

In [20]:
emojis

203      [алина, если, крестьянин, взять, рука, оружие,...
222      [никита, клин, клин, логика, разряд, мы, проти...
229      [анатолий, извинить, вопрос, данный, человек, ...
240      [фима, долго, искать, методичка, толстой, 😒, м...
318      [алина, если, крестьянин, взять, рука, оружие,...
                               ...                        
10040    [дмитрий, ахахахах, шок, 😂😂, сам, б, хотеть, н...
10051             [оля, ссать, речь, идти, 🤣, ещё, плохой]
10063                 [её, сосед, увидеть, весь, район, 😁]
10071    [адам, месяц, платить, налог, жизнь, нищеброд,...
10073     [анастасия, платить, быть, просмотр, готовый, 😄]
Name: comment, Length: 403, dtype: object

Обучение модели

In [21]:
model = Word2Vec(
    min_count=5,
    window=5,
    size=500,
    negative=10,
    min_alpha=0.0007,
    sample=6e-5,
    sg=0)

In [22]:
model.build_vocab(comments)

In [23]:
model.train(comments, total_examples=model.corpus_count, epochs=30, report_delay=1)

(1359497, 2798730)

In [29]:
model.wv.most_similar(positive=["убивать"])

[('соответственно', 0.9333113431930542),
 ('немцов', 0.9117617607116699),
 ('выдумка', 0.8956000208854675),
 ('скрипаль', 0.890803873538971),
 ('политковский', 0.8846331834793091),
 ('литвиненко', 0.8833350539207458),
 ('оккупировать', 0.8830538988113403),
 ('навальный', 0.8770394325256348),
 ('мурза', 0.8766548037528992),
 ('травить', 0.8753408193588257)]

##**Обучение модели на смайлики**

In [ ]:
w2v = Word2Vec(
    min_count=1,
    window=5,
    size=500,
    negative=0,
    min_alpha=0.0007,
    sample=6e-5,
    sg=0)

In [ ]:
w2v.build_vocab(data['filtered'])

In [ ]:
w2v.train(data['filtered'], total_examples=w2v.corpus_count, epochs=30, report_delay=1)

(146817, 255360)

In [ ]:
w2v.wv.most_similar(positive=["флаг"])

[('🧮', 0.1504262387752533),
 ('разочаровать', 0.1478368192911148),
 ('👩🏻\u200d🍳', 0.14491131901741028),
 ('🐎', 0.1445893943309784),
 ('⚔', 0.138372004032135),
 ('🤵🏻', 0.1358640044927597),
 ('пешеход', 0.13075698912143707),
 ('👩\u200d👩\u200d👦', 0.13065919280052185),
 ('🎷', 0.13040149211883545),
 ('букет', 0.1286466121673584)]